<a href="https://colab.research.google.com/github/noam1368/IRProject/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-e235  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
pip install nltk==3.7

Note: you may need to restart the kernel to use updated packages.


In [4]:
#remove mark
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
#remove mark

# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  6 09:28 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [6]:
#remove mark

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [7]:
spark

In [8]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'corpus-big' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh': #todo change when we want to run on the original corpuse
        paths.append(full_path+b.name)

parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd #this is with text becuase we doint the inverted index for the title in this file 

parquetFile.count()


6348910

In [10]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords) #to remark

################################################################################################################ we change it becuase nltk lib didn't work so we won't relate on it for now
# all_stopwords = corpus_stopwords 
################################################################################################################
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS


In [11]:
DL = {}
def process_row(row):
    text = [token.group() for token in RE_WORD.finditer(row['text'].lower())]
    count = len(text)
    print(count)
    #DL[row['id']] = count
    return (row['id'],count)
    
count_rdd = doc_text_pairs.map(process_row)
#DL = count_rdd.collectAsMap()
#print(DL)

IOPub data rate exceeded.                                                       
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
DL = count_rdd.collectAsMap()
DL

{4045403: 1916,
 4045413: 283,
 4045419: 729,
 4045426: 227,
 4045432: 911,
 4045456: 963,
 4045466: 686,
 4045471: 574,
 4045479: 1926,
 4045516: 171,
 4045519: 141,
 4045523: 59,
 4045525: 218,
 4045532: 1485,
 4045544: 675,
 4045546: 38,
 4045554: 780,
 4045575: 247,
 4045577: 1505,
 4045581: 205,
 4045584: 1285,
 4045604: 122,
 4045607: 533,
 4045640: 1262,
 4045651: 88,
 4045659: 561,
 4045661: 464,
 4045672: 1726,
 4045691: 384,
 4045698: 173,
 4045705: 110,
 4045710: 2610,
 4045717: 2134,
 4045722: 807,
 4045725: 267,
 4045760: 261,
 4045770: 429,
 4045791: 228,
 4045792: 183,
 4045815: 193,
 4045829: 691,
 4045840: 627,
 4045849: 1575,
 4045877: 269,
 4045883: 303,
 4045890: 1634,
 4045901: 513,
 4045909: 451,
 4045920: 2179,
 4045928: 155,
 4045953: 141,
 4045974: 370,
 4045978: 1196,
 4045982: 56,
 4045991: 897,
 4046019: 212,
 4046036: 365,
 4046050: 192,
 4046061: 677,
 4046091: 84,
 4046117: 304,
 4046133: 20,
 4046154: 151,
 4046176: 341,
 4046178: 1281,
 4046182: 400,
 4

In [20]:
storage_client = storage.Client()
bucket = storage_client.get_bucket('indexes-my-proj')
blob = bucket.blob('dl.pkl')

with open('dl.pkl','wb') as f:
    pickle.dump(DL,f)
    
index_src = "dl.pkl"
index_dst = f'gs://"indexes-my-proj"/{index_src}'
!gsutil cp $index_src $index_dst

FileNotFoundError: [Errno 2] No such file or directory: 'indexes-my-proj/DL.pkl'

In [ ]:
#DL = pd.DataFrame.from_dict(DL, orient ='index')
#newDL = pd.DataFrame(DL, index=[0])
#newDL.repartition(1).write.csv(f'gs://indexes-my-proj/DL', compression="gzip")

In [ ]:
#newDL.to_csv(f'gs://indexes-my-proj/DL', compression="gzip")

In [ ]:
#newDL.show()

23/01/06 13:42:25 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 17.3 in stage 5.0 (TID 239) (cluster-e235-w-1.c.assignment-3-370813.internal executor 1): TaskKilled (Stage cancelled)
23/01/06 13:42:25 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 5.2 in stage 5.0 (TID 231) (cluster-e235-w-1.c.assignment-3-370813.internal executor 3): TaskKilled (Stage cancelled)
23/01/06 13:42:25 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 7.0 in stage 5.0 (TID 193) (cluster-e235-w-3.c.assignment-3-370813.internal executor 7): TaskKilled (Stage cancelled)
23/01/06 13:42:25 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 16.3 in stage 5.0 (TID 240) (cluster-e235-w-3.c.assignment-3-370813.internal executor 7): TaskKilled (Stage cancelled)
23/01/06 13:42:26 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 21.3 in stage 5.0 (TID 234) (cluster-e235-w-1.c.assignment-3-370813.internal executor 1): TaskKilled (Stage cancelled)
23/01/06 13:42:26 WARN org.